# P2: Multi-Group Campus Tours - Multiagent Pattern

<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together.

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.

**P2: Multi-Group Campus Tours** - Scheduling multiple tour guides to serve independent visitor groups concurrently.

In [ ]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"

In [ ]:
import sys
print("Python version:", sys.version)

In [ ]:
%pip install openai python-dotenv graphviz colorama

In [ ]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")

In [ ]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)

## The Agent Class

First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.

In [ ]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)

In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")

You can also associate tools with the agent. Let's create a tool for writing some string into a file.

In [ ]:
from src.tool_agent.tool import tool

@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")

## The Crew for P2: Multi-Group Campus Tours

In [ ]:
from src.multi_agent.crew import Crew

with Crew() as crew:
    # ---- Group & Guide Setup Agent ---- #
    GS_Agent = Agent(
        name="Group & Guide Setup Agent",
        backstory="You define visitor groups, their sizes, preferred times, and available tour guides with their schedules.",
        task_description="Set up visitor groups (group1: 12 people, preferred time 10:00; group2: 8 people, preferred time 11:00) and available guides (guide1: 9:00-17:00, guide2: 10:00-16:00, guide3: available). Maximum group size per guide is 15.",
        task_expected_output="Structured data: 1) Visitor groups with sizes and preferred times. 2) Available guides with availability windows. 3) Group size constraints (max 15 per guide)."
    )
    
    # ---- Guide Assignment Agent ---- #
    GA_Agent = Agent(
        name="Guide Assignment Agent",
        backstory="You assign guides to visitor groups while balancing workload and minimizing wait times.",
        task_description="Assign guides to groups ensuring: all groups are served, guide availability is respected, group sizes don't exceed 15, and wait times are minimized.",
        task_expected_output="Guide-to-group assignments with start times, ensuring balanced workload across guides."
    )
    
    # ---- Schedule Coordination Agent ---- #
    SC_Agent = Agent(
        name="Schedule Coordination Agent",
        backstory="You coordinate tour schedules to ensure no conflicts and all tours complete within 90 minutes.",
        task_description="Create coordinated tour schedules ensuring each tour completes within 90 minutes, no guide conflicts, and optimal timing.",
        task_expected_output="Coordinated schedule with tour start/end times for each group, respecting guide availability and tour duration constraints."
    )
    
    # ---- Workload Balancing Agent ---- #
    WB_Agent = Agent(
        name="Workload Balancing Agent",
        backstory="You ensure balanced workload distribution across all tour guides.",
        task_description="Verify and adjust assignments to balance the number of tours and total time across all guides.",
        task_expected_output="Balanced workload distribution with equal or near-equal assignment of groups to guides."
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './p2_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    GS_Agent >> GA_Agent >> SC_Agent >> WB_Agent >> Writer_agent

In [ ]:
crew.run()